In [ ]:
import ipywidgets as widgets
import datetime as dt
import pandas as pd
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5]
# reading netcdf from a url without opendap
# from https://towardsdatascience.com/an-efficient-way-to-read-data-from-the-web-directly-into-python-a526a0b4f4cb
# I needed to install h5netcdf
import urllib.request
import xarray as xr
import io

In [ ]:
# Thanks to hatyan https://github.com/Deltares/hatyan for the core of this example
url_prefix="https://nx7384.your-storageshare.de/s" 
station_urls=["45KiDkKGK6SkqCi/download","MK2gNjx8k7qRzsp/download"]
station_names=["Cuxhaven","Brest"] #http://uhslc.soest.hawaii.edu/data/
# using sharingpath in nextcloud
# https://nx7384.your-storageshare.de/apps/sharingpath/wetwin/public/test/series1.csv

In [ ]:
ts_meas=[]
ts_names=[]
print("Downloading data per location:")
for i in np.arange(len(station_urls)):
   myurl=url_prefix+"/"+station_urls[i]
   print("url = "+myurl)
   req = urllib.request.Request(myurl)
   with urllib.request.urlopen(req) as resp:
       ds = xr.open_dataset(io.BytesIO(resp.read()))
       ts_name = station_names[i]
       ts_names.append(ts_name)
       ts_values=ds.sea_level[0,-1000:]/1000
       ts_times=ds.time[-1000:]
       ts_meas.append(pd.DataFrame({'values':ts_values},index=ts_times))
print("Downloads finished")

In [ ]:
# selection of stations with a dropdown list
dd1=widgets.Dropdown(
    options=ts_names,
    value=ts_names[0],
    description='Location',
)
display(dd1)

# create output widget for a plot
out_plot = widgets.Output(layout = widgets.Layout(height='300px'))
 
def update_plot(change):
    choice=dd1.value #or change["new"]
    with out_plot:
        iplot=ts_names.index(choice)
        plt.plot(ts_meas[iplot],color="C0")
        #plt.title(ts_names[iplot])
        plt.title(choice)
        out_plot.clear_output()
        plt.show()

dd1.observe(update_plot,names="value")
display(out_plot)
update_plot(1)
